In [16]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from statsmodels.tsa.stattools import acf, pacf
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

In [3]:
df = pd.read_csv('baseline.csv')
df.head()

,county,is_business,product_type,is_consumption,day,weekday,month,year,segment,sin(dayofyear),cos(dayofyear),sin(hour),cos(hour),eic_count,installed_capacity,temperature,dewpoint,cloudcover_high,cloudcover_low,cloudcover_mid,cloudcover_total,10_metre_u_wind_component,10_metre_v_wind_component,direct_solar_radiation,surface_solar_radiation_downwards,snowfall,total_precipitation,temperature_forecast_local_0h,dewpoint_forecast_local_0h,cloudcover_high_forecast_local_0h,cloudcover_low_forecast_local_0h,cloudcover_mid_forecast_local_0h,cloudcover_total_forecast_local_0h,10_metre_u_wind_component_forecast_local_0h,10_metre_v_wind_component_forecast_local_0h,direct_solar_radiation_forecast_local_0h,surface_solar_radiation_downwards_forecast_local_0h,snowfall_forecast_local_0h,total_precipitation_forecast_local_0h,temperature_forecast_168h,dewpoint_forecast_168h,cloudcover_high_forecast_168h,cloudcover_low_forecast_168h,cloudcover_mid_forecast_168h,cloudcover_total_forecast_168h,10_metre_u_wind_component_forecast_168h,10_metre_v_wind_component_forecast_168h,direct_solar_radiation_forecast_168h,surface_solar_radiation_downwards_forecast_168h,snowfall_forecast_168h,total_precipitation_forecast_168h,temperature_forecast_local_168h,dewpoint_forecast_local_168h,cloudcover_high_forecast_local_168h,cloudcover_low_forecast_local_168h,cloudcover_mid_forecast_local_168h,cloudcover_total_forecast_local_168h,10_metre_u_wind_component_forecast_local_168h,10_metre_v_wind_component_forecast_local_168h,direct_solar_radiation_forecast_local_168h,surface_solar_radiation_downwards_forecast_local_168h,snowfall_forecast_local_168h,total_precipitation_forecast_local_168h,temperature_historical_48h,dewpoint_historical_48h,rain,snowfall_historical_48h,surface_pressure,cloudcover_total_historical_48h,cloudcover_low_historical_48h,cloudcover_mid_historical_48h,cloudcover_high_historical_48h,windspeed_10m,winddirection_10m,shortwave_radiation,direct_solar_radiation_historical_48h,diffuse_radiation,temperature_historical_local_48h,dewpoint_historical_local_48h,rain_historical_local_48h,snowfall_historical_local_48h,surface_pressure_historical_local_48h,cloudcover_total_historical_local_48h,cloudcover_low_historical_local_48h,cloudcover_mid_historical_local_48h,cloudcover_high_historical_local_48h,windspeed_10m_historical_local_48h,winddirection_10m_historical_local_48h,shortwave_radiation_historical_local_48h,direct_solar_radiation_historical_local_48h,diffuse_radiation_historical_local_48h,temperature_historical_168h,dewpoint_historical_168h,rain_historical_168h,snowfall_historical_168h,surface_pressure_historical_168h,cloudcover_total_historical_168h,cloudcover_low_historical_168h,cloudcover_mid_historical_168h,cloudcover_high_historical_168h,windspeed_10m_historical_168h,winddirection_10m_historical_168h,shortwave_radiation_historical_168h,direct_solar_radiation_historical_168h,diffuse_radiation_historical_168h,temperature_historical_local_168h,dewpoint_historical_local_168h,rain_historical_local_168h,snowfall_historical_local_168h,surface_pressure_historical_local_168h,cloudcover_total_historical_local_168h,cloudcover_low_historical_local_168h,cloudcover_mid_historical_local_168h,cloudcover_high_historical_local_168h,windspeed_10m_historical_local_168h,winddirection_10m_historical_local_168h,shortwave_radiation_historical_local_168h,direct_solar_radiation_historical_local_168h,diffuse_radiation_historical_local_168h,temperature_historical_24h,dewpoint_historical_24h,rain_historical_24h,snowfall_historical_24h,surface_pressure_historical_24h,cloudcover_total_historical_24h,cloudcover_low_historical_24h,cloudcover_mid_historical_24h,cloudcover_high_historical_24h,windspeed_10m_historical_24h,winddirection_10m_historical_24h,shortwave_radiation_historical_24h,direct_solar_radiation_historical_24h,diffuse_radiation_historical_24h,target_24h,target_48h,target_72h,target_96h,target_120h,target_144h,target_168h,target_192h,target_216h,target_240h,target_264h,target

In [14]:
list = df.columns.tolist()
print(list)

['county', 'is_business', 'product_type', 'is_consumption', 'day', 'weekday', 'month', 'year', 'segment', 'sin(dayofyear)', 'cos(dayofyear)', 'sin(hour)', 'cos(hour)', 'eic_count', 'installed_capacity', 'temperature', 'dewpoint', 'cloudcover_high', 'cloudcover_low', 'cloudcover_mid', 'cloudcover_total', '10_metre_u_wind_component', '10_metre_v_wind_component', 'direct_solar_radiation', 'surface_solar_radiation_downwards', 'snowfall', 'total_precipitation', 'temperature_forecast_local_0h', 'dewpoint_forecast_local_0h', 'cloudcover_high_forecast_local_0h', 'cloudcover_low_forecast_local_0h', 'cloudcover_mid_forecast_local_0h', 'cloudcover_total_forecast_local_0h', '10_metre_u_wind_component_forecast_local_0h', '10_metre_v_wind_component_forecast_local_0h', 'direct_solar_radiation_forecast_local_0h', 'surface_solar_radiation_downwards_forecast_local_0h', 'snowfall_forecast_local_0h', 'total_precipitation_forecast_local_0h', 'temperature_forecast_168h', 'dewpoint_forecast_168h', 'cloudcove

In [ ]:
import pandas as pd

class DataTransformer:
    def __init__(self, df):
        self.df = df

    def transform(self):
        self.add_season()
        self.add_daypart()
        self.add_feels_like_temperature()
        self.add_total_precipitation()
        self.add_energy_usage_trend()
        self.add_temp_change()
        self.add_prec_change()
        self.add_autocorr_features()
        self.add_energy_price_volatility_and_trend()
        self.perform_clustering()
        self.analyze_transit_and_charging_access()
        return self.df

    def add_season(self):
        def get_season(month):
            if month in [3, 4, 5]:
                return 1 #spring
            elif month in [6, 7, 8]:
                return 2 #summer
            elif month in [9, 10, 11]:
                return 3 #fall
            else:
                return 4 #winter
        
        self.df['season'] = self.df['month'].apply(get_season)

    def add_daypart(self):
        def get_daypart(hour):
            if 5 <= hour < 12:
                return 1 #morning
            elif 12 <= hour < 17:
                return 2 #'Afternoon'
            elif 17 <= hour < 21:
                return 3 #Evening'
            else:
                return 4 #'Night'
        
        self.df['daypart'] = self.df['hour'].apply(get_daypart)

    def add_feels_like_temperature(self):
        def calculate_feels_like(T, u, v):
            wind_speed = (u**2 + v**2)**0.5
            if wind_speed < 4.8:
                return T
            else:
                return 13.12 + 0.6215 * T - 11.37 * (wind_speed ** 0.16) + 0.3965 * T * (wind_speed ** 0.16)

        self.df['feels_like_temp'] = self.df.apply(lambda row: calculate_feels_like(row['temperature'], row['10_metre_u_wind_component'], row['10_metre_v_wind_component']), axis=1)

    def add_total_precipitation(self):
        self.df['total_precipitation'] = self.df['snow'] + self.df['rain'] + self.df['preci']

    def add_energy_usage_trend(self, period = 7):
        self.df['energy_trend'] = self.df['target'].rolling(window = period).mean()

    def add_temp_change(self, interval = 24):
        self.df['temp_change'] = self.df['temperature'].diff(periods = interval)

    def add_prec_change(self):
        self.df['precipitation_change'] = self.df['total_precipitation'].diff()

    def add_autocorr_features(self, lags = 10):
        acf_values = acf(self.df['target'], nlags = lags)
        pacf_values = pacf(self.df['target'], nlags = lags)
        for i in range(lags+1):
            self.df[f'acf_lag_{i}'] = acf_values[i]
            self.df[f'pacf_lag{i}'] = pacf_values[i]

    def add_energy_price_volatility_and_trend(self, window = 7):
        self.df['energy_price_volatility'] = self.df['target'].rolling(window = window).std()

    def perform_clustering(self, n_clusters = 3, features = None):
        if features is None:
            features = ['target_24h', 'target_48h', 'temperature', 'cloudcover_total']
        scaler = StandardScaler()
        scaled_data = scaler.fit_transform(self.df[features])

        kmeans = KMeans(n_clusters=n_clusters, random_state=42)
        self.df['cluster'] = kmeans.fit_predict(scaled_data)

    def analyze_transit_and_charging_access(self):

        results = []
        for consumption_status in [0, 1]:
            subset = self.df[self.df['is_consumption'] == consumption_status]

            # 대중교통 이용률 분석
            business_hours_energy = subset[subset['is_business'] == 1]['target'].mean()
            non_business_hours_energy = subset[subset['is_business'] == 0]['target'].mean()
            transit_usage_estimate = business_hours_energy - non_business_hours_energy

            # 전기차 충전소 접근성 분석
            high_capacity_energy = subset[subset['installed_capacity'] > subset['installed_capacity'].median()]['target'].mean()
            low_capacity_energy = subset[subset['installed_capacity'] <= subset['installed_capacity'].median()]['target'].mean()
            charging_access_estimate = high_capacity_energy - low_capacity_energy

            results.append((consumption_status, transit_usage_estimate, charging_access_estimate))

        # 결과를 하나의 컬럼으로 합침
        for consumption_status, transit_estimate, charging_estimate in results:
            self.df[f'transit_usage_estimate_{consumption_status}'] = transit_estimate
            self.df[f'charging_access_estimate_{consumption_status}'] = charging_estimate



In [ ]:

transformer = DataTransformer(df)
transformed_df = transformer.transform()

# 메모

In [23]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt



#가정: df는 이미 준비된 DataFrame이며, clustering_features는 클러스터링에 사용할 특성 이름 리스트입니다.
clustering_features = ['target_24h', 'target_48h', 'temperature', 'cloudcover_total', 'is_consumption']
data = df[clustering_features]

def calculate_elbow_silhouette(data, range_clusters):
    # 결측치 처리: 바로 위 행의 값으로 대체
    data = data.ffill().bfill()

    inertia = []
    silhouette_scores = []
    
    for n_clusters in range_clusters:
        kmeans = KMeans(n_clusters=n_clusters, random_state=42).fit(data)
        inertia.append(kmeans.inertia_)
        silhouette_scores.append(silhouette_score(data, kmeans.labels_))

    # 엘보우 방법 시각화
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.plot(range_clusters, inertia, marker='o')
    plt.xlabel('Number of clusters')
    plt.ylabel('Inertia')
    plt.title('Elbow Method For Optimal k')

    # 실루엣 점수 시각화
    plt.subplot(1, 2, 2)
    plt.plot(range_clusters, silhouette_scores, marker='o')
    plt.xlabel('Number of clusters')
    plt.ylabel('Silhouette Score')
    plt.title('Silhouette Score For Optimal k')
    plt.show()

def calculate_elbow_silhouette_for_subsets(data, range_clusters, column='is_consumption'):
    subsets = data[column].unique()

    for subset in subsets:
        subset_data = data[data[column] == subset].drop(columns=[column])
        
        # 결측치 처리
        subset_data = subset_data.ffill().bfill()

        # 엘보우 방법 및 실루엣 점수 계산
        print(f"클러스터링 결과 - {column} == {subset}")
        calculate_elbow_silhouette(subset_data, range_clusters)

# 클러스터 수 범위 설정 (예: 2에서 10까지)
range_clusters = range(2, 11)

# 함수 실행
calculate_elbow_silhouette_for_subsets(data, range_clusters)




클러스터링 결과 - is_consumption == 0


/Data/home/limkim/anaconda3/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


KeyboardInterrupt: 